In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from utils import to_grayscale_then_rgb
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split

In [4]:
class ContrastiveLoss:
    def __init__(self, margin=1.0):
        self.margin = margin

    def forward(self, x0, x1, y):
        # Euclidean distance
        diff = x0 - x1
        dist_sq = np.sum(np.power(diff, 2), axis=1)
        dist = np.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = np.clip(mdist, a_min=0.0, a_max=None)
        loss = y * dist_sq + (1 - y) * np.power(dist, 2)
        loss = np.sum(loss) / 2.0 / x0.shape[0]
        return loss

In [5]:
file_data_b_csv=r'/home/jacobo15defrutos/AVS9/4-Red_Siamesa/total_imgs.csv'
df= pd.read_csv(file_data_b_csv)
df_train, df_test = train_test_split(df, test_size=0.2,random_state=42)
print("Train shape: ",df_train.shape)
print("Test shape: ",df_test.shape)


Train shape:  (228, 2)
Test shape:  (57, 2)


In [6]:
labels_train= df_train['Label']
train_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_train['Name']:
    path= folder + name
    img=cv2.imread(path) #,cv2.IMREAD_GRAYSCALE
    img= cv2.resize(img,target_size)
    train_list.append(img)
train= np.array(train_list)

train_fft= np.fft.fftshift(np.fft.fft2(train))
train_abs=np.log(abs(train_fft))
train_n=cv2.normalize(train_abs,None,0, 255, cv2.NORM_MINMAX)
x_train=np.uint8(train_n)
# Convert the original labels to one-hot encoded labels
y_train = np.zeros((labels_train.shape[0], 3))
y_train[np.arange(labels_train.shape[0]), labels_train] = 1

/tmp/ipykernel_43884/1756429171.py:13: RuntimeWarning: divide by zero encountered in log
  train_abs=np.log(abs(train_fft))


In [ ]:

labels_test= df_test['Label']
test_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_test['Name']:
    path= folder + name
    img=cv2.imread(path)#, cv2.IMREAD_GRAYSCALE
    img= cv2.resize(img,target_size)
    test_list.append(img)
test= np.array(test_list)
test_fft= np.fft.fftshift(np.fft.fft2(test))
test_abs=np.log(abs(test_fft))
test_n=cv2.normalize(test_abs,None,0, 255, cv2.NORM_MINMAX)
x_test=np.uint8(test_n)
# Convert the original labels to one-hot encoded labels
y_test = np.zeros((labels_test.shape[0], 3))
y_test[np.arange(labels_test.shape[0]), labels_test] = 1


In [ ]:
# Definir la arquitectura de LeNet-5
input_shape=(224, 224, 1)
loss_fn = ContrastiveLoss()
def build_LeNet5(input_shape, learning_ratea=0.001):
    input_layer = Input(shape=input_shape)

    x = Conv2D(6, kernel_size=(5, 5), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, kernel_size=(5, 5), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    output_layer = Dense(84, activation='relu')(x)
    #output_layer = Dense(3, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_ratea), loss=loss_fn, metrics=['accuracy'])

    return model

model=build_LeNet5(input_shape,0.001)

model.summary()


In [ ]:
learning_rate=0.001
input_shape=(224,224,3)
base_model = VGG19(weights='imagenet',input_shape=input_shape,include_top=True)#,include_top=True
#base_model.get_layer(name='predictions')
# # Remove the original output layer (the 'predictions' layer)
output_layer = Dense(3, activation='softmax', name='custom_output')(base_model.layers[-1].output)
model = Model(inputs=base_model.input, outputs=(output_layer))
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
hist=model.fit(x=x_train, y=y_train, batch_size= 10, validation_split=0.15, epochs=20,verbose=1)

In [ ]:
plt.plot(np.log10(h.history['loss']))
plt.plot(np.log10(h.history['val_loss']), 'r')
plt.legend(['train loss', 'val loss'])

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'], 'r')
plt.legend(['train acc', 'val acc'])
print('train acc: %.2f %% \nval acc: %.2f %%'%(h.history['accuracy'][-1]*100,h.history['val_accuracy'][-1]*100))

In [ ]:
pred=model.predict(x_test,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices

In [ ]:
from sklearn.metrics import accuracy_score
labels=df_test['Label'].astype(int).tolist()
accuracy_score(labels,predicted_class_indices)